# メモ
## このファイルの概要
### フォルダ
2023-07-20_zero_deep_learning_01


## ToDo
- [ ] ~~打ち消し線~~
- [ ]


## 特徴量のアイデア、改善事項、分析事項のメモ


## その他


# 初期設定


## [初期設定]Googleドライブ設定

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## [初期設定]ファイルの読み込み設定
- フォルダについてはフォルダについては'/'で終わるようにする

## フォルダ構成
- base_folder: プロジェクトのフォルダ
-- src: ソースコード
-- data: 入力データ
-- result: 結果、提出データ

In [4]:
glab_folder = '/content/drive/My Drive/Colab/'
project_folder = 'project/2023-07-20_zero_deep_learning_01/'
base_folder = glab_folder + project_folder #プロジェクトのフォルダ

input_folder = base_folder + 'data/input/'
result_folder = base_folder + 'data/output/'

# 実行ファイル名 + タイムスタンプ + 拡張子となる -> タイムスタンプと拡張子は提出ファイル作成時に付加されるようにする必要がある

# プロジェクト概要
- 【第19回_Beginner限定コンペ】国勢調査からの収入予測

## サイト
[https://signate.jp/competitions/576]()

## 結果ファイル
.csv

## スコア



# 関数定義
## ヘッダ
- nt_aiq_xxx_fff

## xxx ->
- table::テーブル作成
- feature::特徴量作成
- analysis:分析、グラフ表示

## 説明テキスト
- [関数] 説明::関数名


# コンペサイト
[https://signate.jp/competitions/488](【AIQuest2021】PBL_01 需要予測・在庫最適化（小売業）)

# コード構成

#### 0. 共通関数呼び出し
- ライブラリ読み込み
- 共通関数読み出し

#### 1. データの読み込み
- inputデータ読み込み

#### 2. 分析（関数作成）
- データ確認、データ分析、グラフ表示
- データ分析、グラフ表示の関数作成

#### 3. 前処理
- NaNデータ補正

#### 4. 特徴量、テーブル生成（関数作成）
- 特徴量生成、特徴量の関数作成
- 新規テーブル生成、新規テーブル生成の関数作成

#### 5. データ整理
- 作成した特徴量、テーブルを結合してtrainとtestのデータを作成する
- 4の特徴量、テーブル生成とかぶる時もありなので、柔軟にする


#### 6. モデリング

#### 7. 提出用ファイルの作成

# 0. 共通関数呼び出し
- ライブラリ読み込み
- 共通関数読み出し

In [5]:
# base
import pandas as pd
import numpy as np

from itertools import product

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

#デバッグ
#import pdb; pdb.set_trace()

In [6]:
pd.__version__

'1.5.3'

In [7]:
# 日本語フォントのインストール
!pip install japanize_matplotlib

import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib
import seaborn as sns

sns.set(font="IPAexGothic")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=35c52233ef575c4ac32caadc08689fde8e171148774b9b8e7d31f35b30952693
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


## [関数]タイムスタンプの取得、生成関数の作成


In [8]:
import datetime

# タイムスタンプ取得関数
def nt_get_now():
  now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  now = now.strftime('%Y%m%d-%H%M%S')

  return now

# タイムスタンプ文字列生成関数
def nt_get_submit_filename(file_name):
  fname = file_name + '['+ nt_get_now() + '].csv'
  print(fname)
  return fname

## [関数]print拡張関数
- nprint()

- globalのnt_debug = True時にprint表示
- False時には表示しない

In [9]:
# 設定
nt_debug = True

In [10]:
def nprint(string):
    if nt_debug == True:
        print(string)

## [設定]headの出力数を変更する

In [11]:
#現在の最大表示列数の出力
nprint(pd.get_option("display.max_columns"))

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

20


## [関数] ラベルのエンコーディング::nt_labal_encoding
- nt_labal_encoding(tbl=pd.DataFrame(), categorical_features=[]):
- テーブル、カラムリストを与えてラベルをint、categoryにエンコーディングする

In [12]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.2 MB/s eta 0:00:00


In [13]:
#@title デフォルトのタイトル テキスト
# ラベルのエンコーディング

# インストール
# !pip install category_encoders

# 使用方法
# list_item = ['商品カテゴリ名']
# tbl = nt_labal_encoding(category_names, list_item)

# 参照
# https://qiita.com/sinchir0/items/b038757e578b790ec96a

# int_change_features = ['pay_year_month','商品カテゴリID','商品カテゴリ名', 'category_main','category_sub','shop_item_id']

def nt_labal_encoding(tbl=pd.DataFrame(), categorical_features=[]):

    # Label Encodingの実施
    import category_encoders as ce

    ce_oe = ce.OrdinalEncoder(cols=categorical_features, handle_unknown='impute')

    #文字を序数に変換
    table_enc = ce_oe.fit_transform(tbl)

    #値を1の始まりから0の始まりにする
    for i in categorical_features:
        table_enc[i] = table_enc[i] - 1

    # カテゴリ変数を全て、category型へ変換
    for i in categorical_features:
        table_enc[i] = table_enc[i].astype('category')

    return table_enc

## [関数] テーブル情報表示関数
nt_aiq_analysis_print_table_info(com=str, tbl=pd.DataFrame())

In [14]:
def nt_aiq_analysis_print_table_info(com=str, tbl=pd.DataFrame()):

    print('###############################################')
    print('# {}'.format(com))
    print('###############################################')

    print('------------------------------Table Info------------------------------')
    print(tbl.info())
    print('-------------------------------------------------------------------------------\n')

    print('------------------------------Table Head------------------------------')
    print(tbl.head())
    print('---------------------------------------------------------------------------\n')

    print('------------------------------Table Describe------------------------------')
    print(tbl.describe())
    print('-------------------------------------------------------------------------------\n')

# 2章パーセプトロン

# 2.3.1

In [15]:
def AND(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.7
    tmp = x1*w1 + x2*w2
    print(tmp)
    if tmp <= theta:
        return 0
    elif tmp >theta:
        return 1

In [16]:
AND(0,0)

0.0


0

In [17]:
AND(1,0)

0.5


0

In [18]:
AND(0,1)

0.5


0

In [19]:
AND(1,1)

1.0


1

## 2.3.2

In [20]:
import numpy as np
x = np.array([0,1])
w = np.array([0.5, 0.5])
b = -0.7
w *x

array([0. , 0.5])

In [21]:
np.sum(w*x)

0.5

In [22]:
np.sum(w*x)+b

-0.19999999999999996

## 2.3.3

In [23]:
def AND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.7
    tmp = np.sum(w*x) + b
    if tmp <=0:
        return 0
    else:
        return 1

In [24]:
def NAND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([-0.5,-0.5])
    b = 0.7
    tmp = np.sum(w*x) + b
    if tmp <=0:
        return 0
    else:
        return 1

In [25]:
def OR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.2
    tmp = np.sum(w*x) + b
    if tmp<0:
        return 0
    else:
        return 1

In [26]:
print(AND(0,0))
print(AND(1,0))
print(AND(0,1))
print(AND(1,1))

0
0
0
1


In [27]:
print(NAND(0,0))
print(NAND(1,0))
print(NAND(0,1))
print(NAND(1,1))


1
1
1
0


In [28]:
print(OR(0,0))
print(OR(1,0))
print(OR(0,1))
print(OR(1,1))


0
1
1
1


## 2.5.2

In [31]:
def XOR(x1, x2):
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y

In [32]:
print(XOR(0,0))
print(XOR(1,0))
print(XOR(0,1))
print(XOR(1,1))

0
1
1
0
